In [8]:
##################Import Modules#######################
import random
import numpy as np
import math
import pandas as pd
import tensorflow as tf
tf.set_random_seed(777) 


##################Define Functions#####################
def five_fold_name(data,i):
    test_names = data[data['index']==i+1]
    
    
def five_fold(data, i):
    test_data = data[data['index']==i+1]
    train_data = data[(data['index']<i+1) | (data['index']>i+1)]
    print(len(test_data), len(train_data))
    
    return train_data , test_data

def set_train_three_layer(repeat, nodes, learning_rate):
    batch_size = 1000
    tf.reset_default_graph()
    keep_prob = tf.placeholder(tf.float32)
    
    X = tf.placeholder(tf.float32, [None, cnt_train])
    Y = tf.placeholder(tf.float32, [None, 2])

    W1 = tf.get_variable( shape= [cnt_train, nodes[0]], name='weight1' , initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([nodes[0]]), name='bias1')
    layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)

    W2 = tf.get_variable(shape =[nodes[0], nodes[1]], name='weight2', initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([nodes[1]]), name='bias2')
    layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
    layer2 = tf.nn.dropout(layer2 , keep_prob=keep_prob)

    W3 = tf.get_variable(shape= [nodes[1], nodes[2]], name='weight3',initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([nodes[2]]), name='bias3')
    layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
    layer3 = tf.nn.dropout(layer3, keep_prob=keep_prob)

    W4 = tf.get_variable(shape=[nodes[2], 2], name='weight4',initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([2]), name='bias4')
    hypothesis = tf.matmul(layer3, W4) + b4



    # cost/loss function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



    # Accuracy computation
    # True if hypothesis>0.5 else False

    predicted = tf.argmax(hypothesis,1)
    correct_prediction = tf.equal(predicted,tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
    with tf.Session() as sess:
        # Initialize TensorFlow variables
        sess.run(tf.global_variables_initializer())

        for step in range(repeat):
            avg_cost = 0
            total_num = int(len(train_x)/batch_size)

            for i in range(total_num):
                batch_x = train_x[i*batch_size:(i+1)*batch_size]
                batch_y = train_y[i*batch_size:(i+1)*batch_size]
                sess.run( train , feed_dict={X: batch_x, Y: batch_y , keep_prob : 0.7})

            if step == repeat-1:
                ####Train Accuracy report####
                train_h, train_p, train_a = sess.run([hypothesis, predicted, accuracy],feed_dict={X: train_x, Y: train_y, keep_prob :0.7})
                print("\nTrain Accuracy: ", train_a)
            if step % 20 == 0 :
                train_h,c, train_p,train_a = sess.run([hypothesis, cost ,predicted, accuracy],feed_dict={X: train_x, Y: train_y, keep_prob :0.7})
                print("\nCurrent Accuracy : ", train_a , "cost : ", c , "Current Step : ", step)
                if train_a > 0.97 :
                    break
        test_h, test_p, test_a = sess.run([hypothesis, predicted, accuracy],feed_dict={X: test_x, Y: test_y, keep_prob :1.0})
        print("\nTest Accuracy: ", test_a)

    return train_p ,train_h, test_p,test_h 

##################READ DATA############################
datafilename = "/Users/taewan/Desktop/Ahn/FinalData_GSM_gene_index_result.csv"
data = pd.read_csv(datafilename)
repeat, layer, node , learning_rate, gene = 100, 3,'1500 1500 1500' ,0.01 , 60
output_directory = '/Users/taewan/Desktop/'



In [13]:
print(data)
for j in range(5):
    nodes = list(map(int , node.split(" ")))
    #####Five fold#####
    train_data, test_data = five_fold(data, j)
    train_GSM = train_data.iloc[:,0]
    test_GSM = test_data.iloc[:,0]
    #####Train Data Set#####
    train_x = train_data.iloc[:,1:-2]
    train_x = train_x.as_matrix()
    train_y = train_data.iloc[:,-1].as_matrix()
    train_y = train_y.flatten()
    train_y = pd.get_dummies(train_y)

    #####Test Data Set#####
    test_x = test_data.iloc[:,1:-2]
    test_x = test_x.as_matrix()
    test_y = test_data.iloc[:,-1].as_matrix()
    test_y = test_y.flatten()
    test_y = pd.get_dummies(test_y)
    print(train_y)
    
    cnt_train = len(train_x[1, :])
    train_p, train_h , test_p ,test_h = (set_train_three_layer(repeat, nodes, learning_rate))
    train_result = pd.concat([train_GSM, train_p], axis=1)
    train_result = pd.concat([train_result, train_h], axis =1 )
    test_result = pd.concat([test_GSM, test_p], axis=1)
    test_result = pd.concat([test_result, test_h], axis=1)
    
    result_train_filename = "result_file_train" + str(j) +".csv"
    train_result.to_csv(output_directory+result_train_filename , sep= ',')
    result_test_filename = "result_file_test" + str(j) +".csv"
    test_result.to_csv(output_directory+result_test_filename , sep= ',')
    ###train h를 file로
    ###test h를 file로 

      Unnamed: 0   XIST  HLA-DRB4 /// LOC100509582  CMPK2   MMP1  RPS4Y1  \
0      GSM253210  3.371                     -0.149 -0.871 -1.346  -0.195   
1      GSM253216  2.780                     -0.126 -1.122 -1.338  -0.037   
2      GSM256745 -0.981                      0.330  2.377 -0.758   3.468   
3      GSM256746 -0.977                      0.112  2.714 -0.796   3.385   
4      GSM256747 -0.993                      0.295  3.745 -0.620   3.251   
5      GSM256748 -1.040                      3.567  1.650  0.179   3.515   
6      GSM256749 -0.973                      3.489  1.610  0.753   3.379   
7      GSM256750 -0.957                      3.228  3.825 -0.047   3.257   
8      GSM257187  1.870                      0.321  0.934 -1.383   1.900   
9      GSM257190  1.410                      0.437  1.166 -1.374   2.342   
10     GSM257248 -1.137                     -0.603  0.917  3.185   2.829   
11     GSM257249 -1.157                     -0.703  0.970  3.185   2.819   
12     GSM25

2432 9481
      0  1
0     1  0
1     1  0
2     1  0
3     1  0
4     1  0
5     1  0
6     1  0
7     1  0
8     0  1
9     0  1
10    1  0
11    1  0
12    1  0
13    1  0
14    1  0
15    0  1
16    0  1
17    0  1
18    0  1
19    0  1
20    0  1
21    0  1
22    0  1
23    0  1
24    0  1
25    0  1
26    0  1
27    0  1
28    0  1
29    0  1
...  .. ..
9451  1  0
9452  1  0
9453  1  0
9454  1  0
9455  1  0
9456  1  0
9457  1  0
9458  1  0
9459  1  0
9460  1  0
9461  1  0
9462  1  0
9463  1  0
9464  1  0
9465  1  0
9466  1  0
9467  1  0
9468  1  0
9469  1  0
9470  1  0
9471  1  0
9472  1  0
9473  1  0
9474  1  0
9475  1  0
9476  1  0
9477  1  0
9478  1  0
9479  1  0
9480  1  0

[9481 rows x 2 columns]

Current Accuracy :  0.518405 cost :  2.9212 Current Step :  0


KeyboardInterrupt: 